In [1]:
import pandas as pd
import numpy as np
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from tensorflow.keras import Sequential, Model, Input
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation




In [2]:
path = "SCUT-FBP5500_v2/train_test_files/split_of_60%training and 40%testing"
train_df = pd.read_csv("%s/train.txt" % (path), sep=" ", names=["image", "score"])
test_df = pd.read_csv("%s/test.txt" % (path), sep=" ", names=["image", "score"])



In [3]:
train_df['purpose'] = 'train'
test_df['purpose'] = 'test'
 
np.random.seed(17) #important for reproducability
val_set_size = int((40*train_df.shape[0])/100)
val_idx = np.random.choice(train_df.shape[0], size=val_set_size)
train_df.loc[val_idx, 'purpose'] = 'validation'
 
df = pd.concat([train_df, test_df])
del train_df, test_df
 
df.sample(5)

,image,score,purpose
1150,CM537.jpg,2.883333,test
2378,AF1558.jpg,3.700000,train
959,AM1502.jpg,2.666667,train
1225,AM1763.jpg,3.750000,test
99,CM726.jpg,3.433333,test


In [4]:
pd.DataFrame(df.purpose.value_counts()).rename(columns = {"purpose": "instances"})
#df.groupby(by=["purpose"])["score"].count()

,instances
train,2219
test,2200
validation,1081


In [20]:
def retrievePixels(img_name):
    path = "SCUT-FBP5500_v2/Images/%s" % (img_name)
    img = load_img(path, grayscale=False, target_size=(224, 224))
    x = img_to_array(img).reshape(1, -1)[0]
    return x
 
df['pixels'] = df['image'].apply(retrievePixels)

In [30]:
import cv2
img_name = "CM148.jpg"
path = "SCUT-FBP5500_v2/Images/%s" % (img_name)
img = load_img(path, grayscale=False, target_size=(224, 224))
x = img_to_array(img).reshape(1, -1)





beauty_model.predict(x)


ValueError: in user code:

    /Users/jsusser/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1462 predict_function  *
        return step_function(self, iterator)
    /Users/jsusser/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1452 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/jsusser/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/jsusser/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/jsusser/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /Users/jsusser/opt/anaconda3/l

In [9]:
train_idx = df[(df['purpose'] == 'train')].index
val_idx = df[(df['purpose'] == 'validation')].index
test_idx = df[(df['purpose'] == 'test')].index

In [10]:
actuals = df.iloc[test_idx].score.values


In [15]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model

beauty_model = load_model("beauty_model.hdf5")

In [ ]:
predictions = beauty_model.predict(features[test_idx])
 


In [ ]:
perf = pd.DataFrame(actuals, columns = ["actuals"])
perf["predictions"] = predictions
print("PC: ",perf[['actuals', 'predictions']].corr(method ='pearson').values[0,1])
print("MAE: ", mean_absolute_error(actuals, predictions))
print("RMSE: ", sqrt(mean_squared_error(actuals, predictions)))